# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [4]:
from statsmodels.formula.api import ols
model = ols(formula="CRIM~INDUS+CHAS+RM+AGE+DIS+PTRATIO+B+LSTAT", data=boston_features).fit()

In [13]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   CRIM   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     145.8
Date:                Fri, 17 May 2019   Prob (F-statistic):          3.63e-125
Time:                        14:48:26   Log-Likelihood:                 339.99
No. Observations:                 506   AIC:                            -662.0
Df Residuals:                     497   BIC:                            -623.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6423      0.080      7.991      0.000       0.484       0.800
INDUS          0.0658      0.009      6.961      0.000       0.047       0.084
CHAS          -0.0064      0.023     -0.285      0.776      -0.051       0.038
RM             0.0216      0.011      1.910      0.057      -0.001       0.044
AGE            0.0171      0.010      1.730      0.084      -0.002       0.036
DIS           -0.3373      0.046     -7.367      0.000      -0.427      -0.247
PTRATIO        0.0208      0.006      3.244      0.001       0.008       0.033
B             -0.1979      0.027     -7.465      0.000      -0.250      -0.146
LSTAT          0.0287      0.010      2.868      0.004       0.009       0.048
==============================================================================
Omnibus:                       22.288   Durbin-Watson:                   0.380
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.011
Skew:                          -0.520   Prob(JB):                     6.11e-06
Kurtosis:                       3.240   Cond. No.                         96.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Run the same model in Scikit-learn

In [10]:
boston_features["CRIM"]

0      0.000000
1      0.153211
2      0.153134
3      0.171005
4      0.250315
5      0.162521
6      0.276046
7      0.327656
8      0.367371
9      0.344658
10     0.373926
11     0.305940
12     0.282362
13     0.481724
14     0.483078
15     0.481329
16     0.535631
17     0.504684
18     0.507126
19     0.496582
20     0.553642
21     0.513370
22     0.552013
23     0.528914
24     0.500052
25     0.511947
26     0.488506
27     0.525396
28     0.503177
29     0.530388
         ...   
476    0.695890
477    0.813789
478    0.773591
479    0.808870
480    0.714587
481    0.712484
482    0.712905
483    0.638603
484    0.620842
485    0.666348
486    0.712182
487    0.695119
488    0.332129
489    0.352559
490    0.365481
491    0.294927
492    0.300311
493    0.346652
494    0.396710
495    0.350028
496    0.400400
497    0.392434
498    0.380349
499    0.349348
500    0.373688
501    0.240099
502    0.206118
503    0.236926
504    0.298671
505    0.210954
Name: CRIM, Length: 506,

In [12]:
from sklearn.linear_model import LinearRegression
X = boston_features
#X.append(np.ones(len(boston_features)))
#X
linreg = LinearRegression().fit(X, boston_features["PTRATIO"])


In [14]:
linreg.coef_

array([-9.54794320e-16, -2.77916748e-17,  2.91580041e-16, -1.12494754e-16,
       -7.84664037e-17,  3.80821403e-16,  1.00000000e+00,  2.69378946e-16,
       -1.76730727e-17,  3.40848776e-17,  4.91818492e-17, -5.16798106e-17,
        9.96680894e-17, -2.78630944e-17])

In [16]:
linreg.intercept_

6.512846035044729e-16

In [15]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   CRIM   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     145.8
Date:                Fri, 17 May 2019   Prob (F-statistic):          3.63e-125
Time:                        14:48:55   Log-Likelihood:                 339.99
No. Observations:                 506   AIC:                            -662.0
Df Residuals:                     497   BIC:                            -623.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6423      0.080      7.991      0.000       0.484       0.800
INDUS          0.0658      0.009      6.961      0.000       0.047       0.084
CHAS          -0.0064      0.023     -0.285      0.776      -0.051       0.038
RM             0.0216      0.011      1.910      0.057      -0.001       0.044
AGE            0.0171      0.010      1.730      0.084      -0.002       0.036
DIS           -0.3373      0.046     -7.367      0.000      -0.427      -0.247
PTRATIO        0.0208      0.006      3.244      0.001       0.008       0.033
B             -0.1979      0.027     -7.465      0.000      -0.250      -0.146
LSTAT          0.0287      0.010      2.868      0.004       0.009       0.048
==============================================================================
Omnibus:                       22.288   Durbin-Watson:                   0.380
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.011
Skew:                          -0.520   Prob(JB):                     6.11e-06
Kurtosis:                       3.240   Cond. No.                         96.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

### Statsmodels

### Scikit-learn

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [21]:
X2 = np.array([.15,6.07, 1,6.1,33.2, 7.6, 17, 383, 10.87, 8, 284])

from statsmodels.tools.tools import add_constant
linreg.predict(add_constant(X2))

ValueError: shapes (11,2) and (14,) not aligned: 2 (dim 1) != 14 (dim 0)

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.